# dog and cat image classifier

In [1]:
#import the modules
import os
import cv2
import numpy as np
import warnings 
warnings.filterwarnings("ignore")



In [2]:
#import zipfile
# # extrcat the data from zip file
# z_path="archive.zip"
# e_path="data_set"

# with zipfile.ZipFile(z_path,"r") as zip_ref:
#     zip_ref.extractall(e_path)
# print("files extract to ",e_path)

# Load and Preprocess  the data

In [33]:
import tensorflow as tf

train_dir = "data_set/train"
val_dir = "data_set/validation"
test_dir = "data_set/test"

img_height, img_width =128,128
batch_size = 32

#train dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    class_names=["cats", "dogs"],   # 0=cats, 1=dogs
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)



# Test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",#give the 0 and 1 value
    label_mode="int",
    class_names=["cats", "dogs"],
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

#normailze the images from 0-255 to 0-1
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Verify
for images, labels in train_ds.take(1):
    print("Batch shape:", images.shape)
    print("Labels:", labels.numpy())


Found 16000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Batch shape: (32, 128, 128, 3)
Labels: [1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 0 1 0 0]


sepate  the validation data  from train_data to  prevnt the overfitting

In [ ]:
import shutil
import random

# Paths
base_dir = "data_set"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "validation")

# Make validation folders
for category in ["cats", "dogs"]:
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)

# Percentage for validation
val_split = 0.2

def move_to_validation(class_name):
    class_train_dir = os.path.join(train_dir, class_name)
    class_val_dir = os.path.join(val_dir, class_name)

    files = [f for f in os.listdir(class_train_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(files)

    val_count = int(len(files) * val_split)
    val_files = files[:val_count]

    print(f"Moving {val_count} {class_name} images to validation...")

    for f in val_files:
        src = os.path.join(class_train_dir, f)
        dst = os.path.join(class_val_dir, f)
        shutil.move(src, dst)   # move instead of copy

# Run for cats and dogs
move_to_validation("cats")
move_to_validation("dogs")

print("Validation folder created with 20% data moved!")


Moving 2000 cats images to validation...
Moving 2000 dogs images to validation...
✅ Validation folder created with 20% data moved!


processe the validation folder

In [15]:
val_ds=tf.keras.utils.image_dataset_from_directory(
    "data_set/validation",
    labels="inferred",
    label_mode="int",
    class_names=["cats","dogs"],
    image_size=(img_height,img_width),
    batch_size=32,
    shuffle=True
)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


Found 4000 files belonging to 2 classes.


In [5]:
import os

print("Train exists:", os.path.exists(train_dir))
print("Test exists:", os.path.exists(test_dir))
print("Train subfolders:", os.listdir(train_dir))


Train exists: True
Test exists: True
Train subfolders: ['cats', 'dogs']


# BUILD THE MODEL

In [6]:
# import the modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.utils import to_categorical


In [32]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")   # binary classification
])


In [22]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
# Summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,449 (28.20 MB)

 Trainable params: 7,392,449 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

train the model

In [26]:
model.fit(train_ds,epochs=10,validation_data=(val_ds))

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 66s 131ms/step - accuracy: 0.5493 - loss: 9.2758 - val_accuracy: 0.5035 - val_loss: 0.6929
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 52s 105ms/step - accuracy: 0.6215 - loss: 0.6245 - val_accuracy: 0.5487 - val_loss: 0.6886
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 53s 105ms/step - accuracy: 0.6913 - loss: 0.5343 - val_accuracy: 0.5620 - val_loss: 0.6842
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 53s 106ms/step - accuracy: 0.7533 - loss: 0.4440 - val_accuracy: 0.5765 - val_loss: 0.6871
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 58s 115ms/step - accuracy: 0.7671 - loss: 0.4300 - val_accuracy: 0.5665 - val_loss: 0.7089
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 63s 126ms/step - accuracy: 0.8071 - loss: 0.3655 - val_accuracy: 0.5577 - val_loss: 0.8579
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 62s 124ms/step - accuracy: 0.8253 - loss: 0.3247 - val_accuracy: 0.5387 - val_loss: 1.0267
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 56s 112ms/step - accuracy: 0.8487 - loss: 0

test the model


In [27]:
loss,accuracy=model.evaluate(train_ds)

500/500 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.8194 - loss: 0.4380


In [28]:
accuracy

0.8193749785423279

save the model

In [31]:
model.save("cat_dog_cnn.h5")